### 직방
- 동이름을 입력하면 아파트 매물을 데이터 프레임으로 만들기
- robots.txt

#### 1. 웹서비스 분석 : URL 찾기, 크롤링 전략 세우기

- 동이름 입력 -> lat, lng
- lat, lng -> geohash : geohash 패키지 설치
- geohash -> item ids
- item ids -> item datas(데이터 프레임)

In [29]:
# 동이름 -> 위도 경도
addr = "성수동"
url = "https://apis.zigbang.com/search?q={}".format(addr)
response = requests.get(url)
data = response.json()["items"][0]
lat, lng = data["lat"], data["lng"]
lat, lng

(37.541725158691406, 127.04335021972656)

In [30]:
# 위도, 경도 -> geohash
# pip install geohash2

In [31]:
import geohash2

In [32]:
# precision : 작을수록 큰범위, 클수록 작은 범위
geohash = geohash2.encode(lat, lng, precision=5)
lat, lng, geohash

(37.541725158691406, 127.04335021972656, 'wydmd')

In [35]:
# geohash -> ids
url = "https://apis.zigbang.com/property/apartments/items?\
domain=zigbang&geohash={}\
&q=type=sales%7Cprice=0~-1%7CfloorArea=0~-1".format(geohash)
response = requests.get(url)
datas = response.json()["items"]
ids = [data["itemId"] for data in datas]
len(ids), ids[:3]

(2345, [17788871, 17813458, 17962259])

In [50]:
# ids -> items
ids_str = str(ids[:200]).replace(" ", "")
url = "https://apis.zigbang.com/property/apartments/items?\
vritemIds={ids}&itemIds={ids}&citemIds={ids}".format(ids=ids_str)
response = requests.get(url)
datas = response.json()["items"]

In [51]:
item_df = pd.DataFrame(datas)

In [52]:
len(item_df.columns)

39

In [53]:
# panda option 설정
pd.options.display.max_columns = len(item_df.columns)

In [54]:
columns = [ "itemId", "buildingFloor", "groupedItemFloor", "grossArea", "lat", "lng", "sales", "itemTitle" ]
item_df = item_df[columns]

In [55]:
item_df["m2"] = item_df["grossArea"].apply(lambda data: data["m2"])
item_df["p"] = item_df["grossArea"].apply(lambda data: data["p"])
item_df.drop(columns=["grossArea"], inplace=True)

In [56]:
item_df.tail(1)

,itemId,buildingFloor,groupedItemFloor,lat,lng,sales,itemTitle,m2,p
199,17331412,15,저층,37.557901,127.027117,87000,"작은방확장, 올수리, 중문설치, 깨끗한집",104,31


#### 함수로 만들기
- 동이름 -> 아파트 매물 데이터 프레임

In [61]:
def crawling_apt(addr):
    # 동이름 -> 위도,경도
    url = "https://apis.zigbang.com/search?q={}".format(addr)
    response = requests.get(url)
    data = response.json()["items"][0]
    lat, lng = data["lat"], data["lng"]
    
    # 위도,경도 -> geohash
    geohash = geohash2.encode(lat, lng, precision=5)
    
    # geohash -> ids
    url = "https://apis.zigbang.com/property/apartments/items?\
domain=zigbang&geohash={}\
&q=type=sales%7Cprice=0~-1%7CfloorArea=0~-1".format(geohash)
    response = requests.get(url)
    datas = response.json()["items"]
    ids = [data["itemId"] for data in datas]
    
    # ids -> items : 200개씩
    dfs = [] 
    for idx in range(0, len(ids), 200):
        start, end = idx, idx + 200
        ids_str = str(ids[start:end]).replace(" ", "")
        url = "https://apis.zigbang.com/property/apartments/items?\
        vritemIds={ids}&itemIds={ids}&citemIds={ids}".format(ids=ids_str)
        response = requests.get(url)
        datas = response.json()["items"]
        item_df = pd.DataFrame(datas)
        columns = [ "itemId", "buildingFloor", "groupedItemFloor", "grossArea", "lat", "lng", "sales", "itemTitle" ]
        item_df = item_df[columns]
        item_df["m2"] = item_df["grossArea"].apply(lambda data: data["m2"])
        item_df["p"] = item_df["grossArea"].apply(lambda data: data["p"])
        item_df.drop(columns=["grossArea"], inplace=True)
        dfs.append(item_df)
    result_df = pd.concat(dfs)
    result_df.reset_index(drop=True, inplace=True)
    return result_df

In [62]:
result = crawling_apt("성수동")
result.tail(2)

,itemId,buildingFloor,groupedItemFloor,lat,lng,sales,itemTitle,m2,p
2343,18414690,21,중층,37.568878,127.031898,85000,집상태 깨끗한 물건,107,32
2344,18408870,25,중층,37.570465,127.024673,118000,시세보다 저렴한 가격으로 투자하기 좋아요 현재 전세거주중,107,32
